In [27]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [14]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

2022/05/30 08:28:03 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [9]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [10]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [11]:
categorical = ['PU_DO', 'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
                           
mean_squared_error(y_train, y_pred, squared = False)

5.898829393942643

### Save Model

In [29]:
import pickle

def save_file(file, name, path):
    with open(os.path.join(path, name), 'wb') as f_out:
        pickle.dump((dv, file), f_out)

save_file(lr, 'lin_reg.bin', './models')

# Experiment Tracking With MLFlow

## Basic Usage

In [15]:
with mlflow.start_run():
    mlflow.set_tag('developer', 'Hayri') #sets developer tag as Hayri

    #set data paths
    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet')

    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared = False)
    
    #log loss metric
    mlflow.log_metric('rmse', rmse)

## XGBoost

In [16]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)

        booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, 'validation')],
            early_stopping_rounds = 50
        )
        
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared = False)

        mlflow.log_metric('rmse', rmse)
    
    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

### Run best parameters with autolog

In [21]:
params = {
    'max_depth': 15,
    'learning_rate': 0.12336302865219267,
    'reg_alpha': 0.04294825818745255,
    'reg_lambda': 0.24351418124417792,
    'min_child_weight': 7.8787970693052465,
    'objective': 'reg:linear',
    'seed': 42
}

with mlflow.start_run():
    mlflow.set_tag('model', 'best_model_autolog')
    mlflow.xgboost.autolog()
    booster = xgb.train(
        params = params,
        dtrain = train,
        num_boost_round = 1000,
        evals = [(valid, 'validation')],
        early_stopping_rounds = 50
    )
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared = False)

    mlflow.log_metric('rmse', rmse)

[10:22:43] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:18.99754
[1]	validation-rmse:17.10143
[2]	validation-rmse:15.46606
[3]	validation-rmse:14.05716
[4]	validation-rmse:12.85140
[5]	validation-rmse:11.82308
[6]	validation-rmse:10.94812
[7]	validation-rmse:10.20475
[8]	validation-rmse:9.58002
[9]	validation-rmse:9.05385
[10]	validation-rmse:8.61408
[11]	validation-rmse:8.24617
[12]	validation-rmse:7.93890
[13]	validation-rmse:7.68163
[14]	validation-rmse:7.46897
[15]	validation-rmse:7.29147
[16]	validation-rmse:7.14439
[17]	validation-rmse:7.02008
[18]	validation-rmse:6.91513
[19]	validation-rmse:6.82877
[20]	validation-rmse:6.75566
[21]	validation-rmse:6.69395
[22]	validation-rmse:6.64088
[23]	validation-rmse:6.59516
[24]	validation-rmse:6.55622
[25]	validation-rmse:6.52312
[26]	validation-rmse:6.49361
[27]	validation-rmse:6.46732
[28]	validation-rmse:6.44434
[29]	validation-rmse:6.42387
[30]	validati

2022/05/30 10:23:23 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/05/30 10:23:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/zoomcampdataengineering/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


## Model Management

## Saving model as artifact

In [26]:
with mlflow.start_run():
    mlflow.set_tag('developer', 'Hayri') #sets developer tag as Hayri

    #set data paths
    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet')

    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared = False)
    
    #log loss metric
    mlflow.log_metric('rmse', rmse)

    save_file(lr, 'lin_reg.bin', './models') #Save model to directory before logging as artifact

    mlflow.log_artifact(local_path = 'models/lin_reg.bin', artifact_path = 'models_pickle')

In [23]:
# To avoid autolog model save
mlflow.xgboost.autolog(disable=True)

## Save model with log_model
**and save preprocessor**

In [30]:
params = {
    'max_depth': 15,
    'learning_rate': 0.12336302865219267,
    'reg_alpha': 0.04294825818745255,
    'reg_lambda': 0.24351418124417792,
    'min_child_weight': 7.8787970693052465,
    'objective': 'reg:linear',
    'seed': 42
}

with mlflow.start_run():
    mlflow.set_tag('model', 'best_model_logging')
    mlflow.log_params(params)
    booster = xgb.train(
        params = params,
        dtrain = train,
        num_boost_round = 1000,
        evals = [(valid, 'validation')],
        early_stopping_rounds = 50
    )
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared = False)
    mlflow.log_metric('rmse', rmse)

    #save preprocessor
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact(local_path = './models/preprocessor.b', artifact_path = 'preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path = 'models_mlflow')

[11:34:00] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:18.99754
[1]	validation-rmse:17.10143
[2]	validation-rmse:15.46606
[3]	validation-rmse:14.05716
[4]	validation-rmse:12.85140
[5]	validation-rmse:11.82308
[6]	validation-rmse:10.94812
[7]	validation-rmse:10.20475
[8]	validation-rmse:9.58002
[9]	validation-rmse:9.05385
[10]	validation-rmse:8.61408
[11]	validation-rmse:8.24617
[12]	validation-rmse:7.93890
[13]	validation-rmse:7.68163
[14]	validation-rmse:7.46897
[15]	validation-rmse:7.29147
[16]	validation-rmse:7.14439
[17]	validation-rmse:7.02008
[18]	validation-rmse:6.91513
[19]	validation-rmse:6.82877
[20]	validation-rmse:6.75566
[21]	validation-rmse:6.69395
[22]	validation-rmse:6.64088
[23]	validation-rmse:6.59516
[24]	validation-rmse:6.55622
[25]	validation-rmse:6.52312
[26]	validation-rmse:6.49361
[27]	validation-rmse:6.46732
[28]	validation-rmse:6.44434
[29]	validation-rmse:6.42387
[30]	validati

## Making predictions using information from MLflow artifact

In [33]:
logged_model = 'runs:/250fe19f833b48f9badcd00b4e532ef9/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Load model as a xgboost model
xgboost_loaded_model = mlflow.xgboost.load_model(logged_model)

print(f'mlflow pyFuncModel type = {type(loaded_model)}\n')
print(f'xgboost model type = {type(xgboost_loaded_model)}')

[11:39:55] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[11:39:55] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
mlflow pyFuncModel type = <class 'mlflow.pyfunc.PyFuncModel'>

xgboost model type = <class 'xgboost.core.Booster'>


## Train models for model registry

In [34]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

/home/zoomcampdataengineering/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
